In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import h5py
import sys,os,time,json
import random
from IPython.display import display
import fidle

from myutils.dataset import  read_dataset
from myutils.models import get_model_v1,get_model_v2,get_model_v3

In [13]:
enhanced_dir = f'./data'
# enhanced_dir = f'{datasets_dir}/GTSRB/enhanced'

# ---- For tests
datasets      = ['set-24x24-L', 'set-24x24-RGB', 'set-48x48-RGB']
models        = {'v1':'get_model_v1', 'v2':'get_model_v2', 'v3':'get_model_v3'}
batch_size    = 64
epochs        = 20
scale         = 1
with_datagen  = False
fit_verbosity = 0
run_dir ='./full_out1'

In [14]:
random.seed(time.time())

# ---- Where I am ?
now    = time.strftime("%A %d %B %Y - %Hh%Mm%Ss")
here   = os.getcwd()
tag_id = '{:06}'.format(random.randint(0,99999))



print('Full Convolutions Notebook :')
print('  Now is             : ', now      )
print('  Tag id             : ', tag_id   )
print('  Working directory  : ', here     )
print('  Output  directory  : ', run_dir  )
print('  for tensorboard    : ', f'--logdir {run_dir}/logs')

Full Convolutions Notebook :
  Now is             :  Monday 11 December 2023 - 09h50m12s
  Tag id             :  096005
  Working directory  :  /home/20015587/Documents/ING3/DeepL/TP1/resolution
  Output  directory  :  ./full_out1
  for tensorboard    :  --logdir ./full_out1/logs


In [15]:
def multi_run(enhanced_dir, datasets, models, datagen=None,
              scale=1, batch_size=64, epochs=16, 
              fit_verbosity=0, tag_id='last'):
    """
    Launches a dataset-model combination
    args:
        enhanced_dir   : Directory of the enhanced datasets
        datasets       : List of dataset (whitout .h5)
        models         : List of model like { "model name":get_model(), ...}
        datagen        : Data generator or None (None)
        scale          : % of dataset to use.  1 mean all. (1)
        batch_size     : Batch size (64)
        epochs         : Number of epochs (16)
        fit_verbosity  : Verbose level (0)
        tag_id         : postfix for report, logs and models dir (_last)
    return:
        report        : Report as a dict for Pandas.
    """  
    # ---- Logs and models dir
    #
    os.makedirs(f'{run_dir}/logs_{tag_id}',   mode=0o750, exist_ok=True)
    os.makedirs(f'{run_dir}/models_{tag_id}', mode=0o750, exist_ok=True)
    
    # ---- Columns of output
    #
    output={}
    output['Dataset'] = []
    output['Size']    = []
    for m in models:
        output[m+'_Accuracy'] = []
        output[m+'_Duration'] = []

    # ---- Let's go
    #
    for d_name in datasets:
        print("\nDataset : ",d_name)

        # ---- Read dataset
        x_train,y_train,x_test,y_test = read_dataset(enhanced_dir, d_name)
        output['Dataset'].append(d_name)
        output['Size'].append(0)
        
        # ---- Rescale
        x_train,y_train,x_test,y_test = fidle.utils.rescale_dataset(x_train,y_train,x_test,y_test, scale=scale)
        
        # ---- Get the shape
        (n,lx,ly,lz) = x_train.shape

        # ---- For each model
        for m_name,m_function in models.items():
            print("    Run model {}  : ".format(m_name), end='')
            # ---- get model
            try:
                # ---- get function by name
                m_function=globals()[m_function]
                model=m_function(lx,ly,lz)
                # ---- Compile it
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                # ---- Callbacks tensorboard
                log_dir = f'{run_dir}/logs_{tag_id}/tb_{d_name}_{m_name}'
                tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
                # ---- Callbacks bestmodel
                save_dir = f'{run_dir}/models_{tag_id}/model_{d_name}_{m_name}.h5'
                bestmodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_dir, verbose=0, monitor='accuracy', save_best_only=True)
                # ---- Train
                start_time = time.time()
                if datagen==None:
                    # ---- No data augmentation (datagen=None) --------------------------------------
                    history = model.fit(x_train, y_train,
                                        batch_size      = batch_size,
                                        epochs          = epochs,
                                        verbose         = fit_verbosity,
                                        validation_data = (x_test, y_test),
                                        callbacks       = [tensorboard_callback, bestmodel_callback])
                else:
                    # ---- Data augmentation (datagen given) ----------------------------------------
                    datagen.fit(x_train)
                    history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                                        steps_per_epoch = int(len(x_train)/batch_size),
                                        epochs          = epochs,
                                        verbose         = fit_verbosity,
                                        validation_data = (x_test, y_test),
                                        callbacks       = [tensorboard_callback, bestmodel_callback])
                    
                # ---- Result
                end_time = time.time()
                duration = end_time-start_time
                accuracy = max(history.history["val_accuracy"])*100
                #
                output[m_name+'_Accuracy'].append(accuracy)
                output[m_name+'_Duration'].append(duration)
                print(f"Accuracy={accuracy: 7.2f}    Duration={duration: 7.2f}")
            except:
                print('An error occured for :',m_name)
                output[m_name+'_Accuracy'].append('0')
                output[m_name+'_Duration'].append('999')
                print('-')
    return output

In [16]:
chrono=fidle.Chrono()
chrono.start()

print('\n---- Run','-'*50)


# ---- Data augmentation or not
#
if with_datagen :
    datagen = keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
                                                           featurewise_std_normalization=False,
                                                           width_shift_range=0.1,
                                                           height_shift_range=0.1,
                                                           zoom_range=0.2,
                                                           shear_range=0.1,
                                                           rotation_range=10.)
else:
    datagen=None
    
# ---- Run
#
output = multi_run(enhanced_dir,
                   datasets, 
                   models,
                   datagen       = datagen,
                   scale         = scale,
                   batch_size    = batch_size,
                   epochs        = epochs,
                   fit_verbosity = fit_verbosity,
                   tag_id        = tag_id)

# ---- Save report
#
report={}
report['output']=output
report['description'] = f'scale={scale} batch_size={batch_size} epochs={epochs} data_aug={with_datagen}'

report_name=f'{run_dir}/report_{tag_id}.json'

with open(report_name, 'w') as file:
    json.dump(report, file, indent=4)

print('\nReport saved as ',report_name)

chrono.show()
print('-'*59)



---- Run --------------------------------------------------

Dataset :  set-24x24-L
(932, 24, 24, 1) (932,)
Datasets have been shuffled.
Datasets have been resized with a factor  1
    Run model v1  : Accuracy=  60.52    Duration=  43.88
    Run model v2  : Accuracy=  37.77    Duration=  66.08
    Run model v3  : Accuracy=  21.03    Duration=  59.14

Dataset :  set-24x24-RGB
(932, 24, 24, 3) (932,)
Datasets have been shuffled.
Datasets have been resized with a factor  1
    Run model v1  : Accuracy=  66.09    Duration=  47.45
    Run model v2  : Accuracy=  52.36    Duration=  67.74
    Run model v3  : Accuracy=  33.48    Duration=  60.18

Dataset :  set-48x48-RGB
(932, 48, 48, 3) (932,)
Datasets have been shuffled.
Datasets have been resized with a factor  1
    Run model v1  : Accuracy=  64.81    Duration= 125.81
    Run model v2  : Accuracy=  60.52    Duration= 154.00
    Run model v3  : Accuracy=  36.05    Duration= 127.51

Report saved as  ./full_out1/report_096005.json
Duration :